In [26]:
import pandas as pd
# df = pd.read_csv('Data/final.csv')
df = pd.read_csv('final.csv')
df.head()

,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,amount,avg_txn_amount,errorBalanceDest,balanceDiffDest,isFlaggedFraud,log_amount,hour,zero_balance_flag,isFraud
0,0.0,0.0,0.0,1.0,0.0,9839.64,9839.64,9839.64,-9839.64,0,9.194276,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,1864.28,1864.28,1864.28,-1864.28,0,7.531166,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,181.00,181.00,181.00,-181.00,0,5.204007,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,181.00,181.00,21363.00,-21363.00,0,5.204007,1.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,11668.14,11668.14,11668.14,-11668.14,0,9.364703,1.0,0.0,0.0


In [27]:
df.shape

(1285009, 14)

In [28]:
df.isna().sum()

,0
type_CASH_IN,0
type_CASH_OUT,0
type_DEBIT,0
type_PAYMENT,0
type_TRANSFER,0
amount,0
avg_txn_amount,0
errorBalanceDest,0
balanceDiffDest,0
isFlaggedFraud,0


In [29]:
df = df.dropna(axis=0)
df.isna().sum()

,0
type_CASH_IN,0
type_CASH_OUT,0
type_DEBIT,0
type_PAYMENT,0
type_TRANSFER,0
amount,0
avg_txn_amount,0
errorBalanceDest,0
balanceDiffDest,0
isFlaggedFraud,0


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(df.drop(['isFraud'],axis=1))
y = df['isFraud']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
!pip install tensorflow scikit-learn

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Nadam
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import tensorflow as tf

# ------------------------------------------------------------
# Compute class weights to handle imbalance
# ------------------------------------------------------------
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

# ------------------------------------------------------------
# Optional: Focal loss for imbalanced datasets
# ------------------------------------------------------------
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        loss = -y_true * alpha * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
               - (1 - y_true) * (1 - alpha) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return focal_loss_fixed

# ------------------------------------------------------------
# ANN model tuned for large, imbalanced binary classification
# ------------------------------------------------------------
model = Sequential([
    Dense(512, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(64, activation='relu'),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
])

# Compile with Nadam optimizer and focal loss (can switch to binary_crossentropy)
model.compile(
    optimizer=Nadam(learning_rate=0.001),
    loss=focal_loss(gamma=2., alpha=0.25),  # try changing to 'binary_crossentropy' if unstable
    metrics=['accuracy']
)

# ------------------------------------------------------------
# Train the model with class weights
# ------------------------------------------------------------
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=4096,   # better for large dataset
    validation_split=0.1,
    class_weight=class_weights,
    verbose=1
)

# ------------------------------------------------------------
# Evaluate on test set
# ------------------------------------------------------------
y_pred_probs = model.predict(X_test, batch_size=4096)

# Find the best threshold for F1 score
best_f1 = 0
best_thresh = 0.5
for t in np.arange(0.1, 0.99, 0.01):
    preds = (y_pred_probs > t).astype(int)
    f1 = f1_score(y_test, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

# Predict with the best threshold
y_pred = (y_pred_probs > best_thresh).astype(int)

# ------------------------------------------------------------
# Final metrics
# ------------------------------------------------------------
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\nBest Threshold: {best_thresh:.2f}")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score:  {f1:.4f}")


Class Weights: {0: np.float64(0.5005974995593014), 1: np.float64(418.91035044824775)}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
226/226 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - accuracy: 0.8154 - loss: 0.1071 - val_accuracy: 0.9987 - val_loss: 0.0047
Epoch 2/15
226/226 ━━━━━━━━━━━━━━━━━━━━ 46s 205ms/step - accuracy: 0.9988 - loss: 0.0035 - val_accuracy: 0.9988 - val_loss: 0.0011
Epoch 3/15
226/226 ━━━━━━━━━━━━━━━━━━━━ 83s 209ms/step - accuracy: 0.9989 - loss: 0.0012 - val_accuracy: 0.9989 - val_loss: 6.6856e-04
Epoch 4/15
 90/226 ━━━━━━━━━━━━━━━━━━━━ 26s 194ms/step - accuracy: 0.9989 - loss: 9.1178e-04

In [48]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

8032/8032 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step


In [49]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [58]:
from sklearn.metrics import recall_score, f1_score, accuracy_score

In [59]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.9991712126753878

In [60]:
recall = recall_score(y_test,y_pred)
recall

0.36507936507936506

In [61]:
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


F1 Score: 0.5191873589164786


In [64]:
from sklearn.metrics import f1_score
import numpy as np

best_f1 = 0
best_thresh = 0

for t in np.arange(0.1, 0.9, 0.01):
    preds = (y_pred_probs > t).astype(int)
    f1 = f1_score(y_test, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best Threshold: {best_thresh:.2f}, Best F1: {best_f1:.4f}")

# Apply optimal threshold
y_pred = (y_pred_probs > best_thresh).astype(int)


Best Threshold: 0.89, Best F1: 0.2034


In [65]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

best_f1 = 0
best_thresh = 0

for t in np.arange(0.1, 0.95, 0.01):
    preds = (y_pred_probs > t).astype(int)
    f1 = f1_score(y_test, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

# Final metrics at optimal threshold
final_preds = (y_pred_probs > best_thresh).astype(int)

acc = accuracy_score(y_test, final_preds)
recall = recall_score(y_test, final_preds)
precision = precision_score(y_test, final_preds)
f1 = f1_score(y_test, final_preds)

print(f"Best Threshold: {best_thresh:.2f}")
print(f"Accuracy: {acc:.4f}")
print(f"Recall:   {recall:.4f}")
print(f"Precision:{precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Best Threshold: 0.94
Accuracy: 0.9943
Recall:   0.7810
Precision:0.1505
F1 Score: 0.2523
